# INITIAL TEST

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from dataclasses import dataclass
from typing import Callable, List, Dict, Optional, Tuple
import os
import json
import time
from datetime import datetime
import pandas as pd
import psutil



class OptimizationResult:
    """Enhanced optimization result storage"""
    def __init__(self, **kwargs):
        self.x_final = kwargs.get('x_final')
        self.f_final = kwargs.get('f_final')
        self.success = kwargs.get('success')
        self.iterations = kwargs.get('iterations')
        self.runtime = kwargs.get('runtime')
        self.path = kwargs.get('path', [])
        self.f_path = kwargs.get('f_path', [])
        self.grad_norm_path = kwargs.get('grad_norm_path', [])
        self.timestamps = kwargs.get('timestamps', [])
        self.memory_usage = kwargs.get('memory_usage', [])
        self.flops_per_step = kwargs.get('flops_per_step', [])
        self.method = kwargs.get('method')
        self.dimension = kwargs.get('dimension')
        self.function_name = kwargs.get('function_name')
        self.x_initial = kwargs.get('x_initial')
        self.f_initial = kwargs.get('f_initial')
        self.grad_initial = kwargs.get('grad_initial')
        self.grad_final = kwargs.get('grad_final')

        # Calculate distance from global minimum
        x_min, f_min = TestFunctions.get_global_minimum(self.function_name, self.dimension)
        if x_min is not None and f_min is not None:
            self.distance_to_minimum = np.linalg.norm(self.x_final - x_min)
            self.f_error = abs(self.f_final - f_min)
        else:
            self.distance_to_minimum = None
            self.f_error = None


class FLOPCounter:
    """Tracks floating point operations"""
    def __init__(self):
        self.flops = 0
        self.operation_counts = {
            'add': 0,
            'multiply': 0,
            'divide': 0,
            'sqrt': 0,
            'exp': 0,
            'log': 0,
            'trig': 0
        }

    def add_flops(self, operation: str, count: int = 1):
        self.operation_counts[operation] += count
        # Update total FLOPS based on operation weight
        weights = {
            'add': 1,
            'multiply': 1,
            'divide': 4,
            'sqrt': 8,
            'exp': 10,
            'log': 10,
            'trig': 15
        }
        self.flops += weights[operation] * count

    def get_summary(self) -> dict:
        return {
            'total_flops': self.flops,
            'operations': self.operation_counts
        }

class TestFunctions:
    """Test functions that work with any dimension"""
    @staticmethod
    def get_global_minimum(func_name: str, dimension: int = 2) -> tuple:
        """Get global minimum for a given function and dimension"""
        global_minima = {
            'ackley': (np.zeros(dimension), 0.0),
            'rastrigin': (np.zeros(dimension), 0.0),
            'rosenbrock': (np.ones(dimension), 0.0),
            'sphere': (np.zeros(dimension), 0.0),
            'michalewicz': (None, None),  # Varies with dimension
        }
        return global_minima.get(func_name, (None, None))

    @staticmethod
    def ackley(x: np.ndarray) -> float:
        """Ackley function for n dimensions"""
        n = len(x)
        sum_sq = np.sum(x**2)
        sum_cos = np.sum(np.cos(2 * np.pi * x))
        return (-20 * np.exp(-0.2 * np.sqrt(sum_sq / n))
                - np.exp(sum_cos / n)
                + 20 + np.e)

    @staticmethod
    def ackley_gradient(x: np.ndarray) -> np.ndarray:
        """Gradient of Ackley function"""
        n = len(x)
        sum_sq = np.sum(x**2)
        sum_cos = np.sum(np.cos(2 * np.pi * x))

        term1 = (20 * 0.2 / np.sqrt(n * sum_sq)) * np.exp(-0.2 * np.sqrt(sum_sq / n)) * x
        term2 = (2 * np.pi / n) * np.exp(sum_cos / n) * np.sin(2 * np.pi * x)
        return term1 + term2

    @staticmethod
    def ackley_hessian(x: np.ndarray) -> np.ndarray:
        """Numerical approximation of Ackley Hessian"""
        eps = 1e-8
        n = len(x)
        H = np.zeros((n, n))
        grad = TestFunctions.ackley_gradient

        for i in range(n):
            for j in range(n):
                x_ij = x.copy()
                x_ij[i] += eps
                x_ij[j] += eps
                H[i,j] = (grad(x_ij)[i] - grad(x)[i]) / eps

        return (H + H.T) / 2  # Ensure symmetry

    @staticmethod
    def rastrigin(x: np.ndarray) -> float:
        """Rastrigin function for n dimensions"""
        n = len(x)
        return 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

    @staticmethod
    def rastrigin_gradient(x: np.ndarray) -> np.ndarray:
        """Gradient of Rastrigin function"""
        return 2 * x + 20 * np.pi * np.sin(2 * np.pi * x)

    @staticmethod
    def rastrigin_hessian(x: np.ndarray) -> np.ndarray:
        """Hessian of Rastrigin function"""
        n = len(x)
        return 2 * np.eye(n) + 40 * np.pi**2 * np.diag(np.cos(2 * np.pi * x))

    @staticmethod
    def sphere(x: np.ndarray) -> float:
        """Sphere function for n dimensions"""
        return np.sum(x**2)

    @staticmethod
    def sphere_gradient(x: np.ndarray) -> np.ndarray:
        """Gradient of Sphere function"""
        return 2 * x

    @staticmethod
    def sphere_hessian(x: np.ndarray) -> np.ndarray:
        """Hessian of Sphere function"""
        n = len(x)
        return 2 * np.eye(n)

    @staticmethod
    def rosenbrock(x: np.ndarray) -> float:
        """Rosenbrock function for n dimensions"""
        return np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

    @staticmethod
    def rosenbrock_gradient(x: np.ndarray) -> np.ndarray:
        """Gradient of Rosenbrock function"""
        n = len(x)
        grad = np.zeros(n)
        grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
        grad[-1] = 200 * (x[-1] - x[-2]**2)
        if n > 2:
            grad[1:-1] = 200 * (x[1:-1] - x[:-2]**2) - 400 * x[1:-1] * (x[2:] - x[1:-1]**2) - 2 * (1 - x[1:-1])
        return grad

    @staticmethod
    def rosenbrock_hessian(x: np.ndarray) -> np.ndarray:
        """Numerical approximation of Rosenbrock Hessian"""
        eps = 1e-8
        n = len(x)
        H = np.zeros((n, n))
        grad = TestFunctions.rosenbrock_gradient

        for i in range(n):
            for j in range(n):
                x_ij = x.copy()
                x_ij[i] += eps
                x_ij[j] += eps
                H[i,j] = (grad(x_ij)[i] - grad(x)[i]) / eps

        return (H + H.T) / 2  # Ensure symmetry

class OptimizationLogger:
    """Handles logging of optimization progress"""
    def __init__(self, method: str, function_name: str, dimension: int):
        self.method = method
        self.function_name = function_name
        self.dimension = dimension
        self.reset()

    def reset(self):
        self.path = []
        self.f_path = []
        self.grad_norm_path = []
        self.step_sizes = []
        self.memory_usage = []
        self.timestamps = []
        self.start_time = time.time()

    def log_iteration(self, x: np.ndarray, f: float, grad_norm: float, step_size: float):
        self.path.append(x.copy())
        self.f_path.append(f)
        self.grad_norm_path.append(grad_norm)
        self.step_sizes.append(step_size)
        self.memory_usage.append(self.get_memory_usage())
        self.timestamps.append(time.time() - self.start_time)

    @staticmethod
    def get_memory_usage() -> float:
        """Get current memory usage in MB"""
        import psutil
        process = psutil.Process()
        return process.memory_info().rss / 1024 / 1024

    def save_logs(self, base_dir: str):
        """Save optimization logs to CSV"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_dir = os.path.join(base_dir, self.function_name, str(self.dimension) + "D", self.method)
        os.makedirs(log_dir, exist_ok=True)

        log_data = {
            'iteration': range(len(self.path)),
            'function_value': self.f_path,
            'gradient_norm': self.grad_norm_path,
            'step_size': self.step_sizes,
            'memory_mb': self.memory_usage,
            'runtime_seconds': self.timestamps
        }

        # Add parameter values
        for i in range(self.dimension):
            log_data[f'x{i+1}'] = [p[i] for p in self.path]

        df = pd.DataFrame(log_data)
        df.to_csv(os.path.join(log_dir, f'optimization_log_{timestamp}.csv'), index=False)

class Visualizer:
    """Enhanced visualization capabilities"""
    @staticmethod
    def plot_optimization_summary(results: Dict[str, OptimizationResult], save_dir: str, function_name: str):
        """Plot summary comparing initial and final states"""
        if not results:
            return

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

        methods = list(results.keys())
        x = np.arange(len(methods))

        # Fix the ticks warning by setting them explicitly
        for ax in [ax1, ax2, ax3, ax4]:
            ax.set_xticks(x)
            ax.set_xticklabels(methods, rotation=45)

        # Function Values Plot
        initial_values = [result.f_initial for result in results.values()]
        final_values = [result.f_final for result in results.values()]
        width = 0.35

        ax1.bar(x - width/2, initial_values, width, label='Initial', color='lightcoral')
        ax1.bar(x + width/2, final_values, width, label='Final', color='lightgreen')
        ax1.set_ylabel('Function Value')
        ax1.set_title('Initial vs Final Function Values')
        ax1.legend()
        ax1.grid(True)

        # Add global minimum line if available
        _, f_min = TestFunctions.get_global_minimum(function_name, results[methods[0]].dimension)
        if f_min is not None:
            ax1.axhline(y=f_min, color='r', linestyle='--', label=f'Global Min ({f_min})')
            ax1.legend()

        # Gradient Norms Plot
        initial_grads = [np.linalg.norm(result.grad_initial) for result in results.values()]
        final_grads = [np.linalg.norm(result.grad_final) for result in results.values()]

        ax2.bar(x - width/2, initial_grads, width, label='Initial', color='lightcoral')
        ax2.bar(x + width/2, final_grads, width, label='Final', color='lightgreen')
        ax2.set_ylabel('Gradient Norm')
        ax2.set_title('Initial vs Final Gradient Norms')
        ax2.legend()
        ax2.grid(True)

        # Runtime Comparison
        runtimes = [result.runtime for result in results.values()]
        ax3.bar(methods, runtimes, color='skyblue')
        ax3.set_ylabel('Runtime (seconds)')
        ax3.set_title('Total Runtime by Method')
        ax3.grid(True)

        # Iterations Comparison
        iterations = [result.iterations for result in results.values()]
        ax4.bar(methods, iterations, color='lightgreen')
        ax4.set_ylabel('Number of Iterations')
        ax4.set_title('Total Iterations by Method')
        ax4.grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f'optimization_summary_{function_name}.png'), dpi=300, bbox_inches='tight')
        plt.close()

    @staticmethod
    def plot_2d_trajectory(f: Callable, result: OptimizationResult, save_dir: str, experiment_num: int = None):
        """Plot optimization trajectory for 2D problems with multi-scale contours"""
        if result.dimension != 2:
            return

        plt.figure(figsize=(12, 10))

        # Get path bounds and include (0,0)
        path = np.array(result.path)
        x_min_traj = min(float(path[:, 0].min()), 0)
        x_max_traj = max(float(path[:, 0].max()), 0)
        y_min_traj = min(float(path[:, 1].min()), 0)
        y_max_traj = max(float(path[:, 1].max()), 0)

        # Add margin to bounds
        margin = 0.1
        x_range = x_max_traj - x_min_traj
        y_range = y_max_traj - y_min_traj
        plot_x_min = x_min_traj - margin * x_range
        plot_x_max = x_max_traj + margin * x_range
        plot_y_min = y_min_traj - margin * y_range
        plot_y_max = y_max_traj + margin * y_range

        # Create contour plot covering the full trajectory
        x = np.linspace(plot_x_min, plot_x_max, 200)
        y = np.linspace(plot_y_min, plot_y_max, 200)
        X, Y = np.meshgrid(x, y)
        Z = np.array([[f(np.array([xi, yi])) for xi in x] for yi in y])

        # Create contour levels starting from 0
        global_max = float(Z.max())
        global_levels = np.linspace(0, global_max, 50)

        # Plot contours
        plt.contour(X, Y, Z, levels=global_levels, cmap='viridis', alpha=0.7)
        plt.colorbar(label='Function Value', ticks=np.linspace(0, global_max, 10))

        # Plot trajectory
        plt.plot(path[:, 0], path[:, 1], 'r.-', label='Optimization Path', linewidth=1, markersize=2)
        plt.plot(path[0, 0], path[0, 1], 'go', label='Start', markersize=8)
        plt.plot(path[-1, 0], path[-1, 1], 'ro', label='End', markersize=8)

        # Get and plot global minimum
        x_min, f_min = TestFunctions.get_global_minimum(result.function_name)
        if x_min is not None:
            plt.plot(x_min[0], x_min[1], 'k*', label='Global Minimum', markersize=10)

        # Set axis limits to show full trajectory (using scalar values)
        plt.xlim(float(plot_x_min), float(plot_x_max))
        plt.ylim(float(plot_y_min), float(plot_y_max))

        # Add gridlines
        plt.grid(True)

        # Labels and title
        plt.title(f'{result.function_name} - {result.method}\n'
                f'Final value: {result.f_final:.6f}\n'
                f'Iterations: {result.iterations}')
        plt.xlabel('x₁')
        plt.ylabel('x₂')
        plt.legend()

        # Include experiment number in filename if provided
        experiment_suffix = f'_exp{experiment_num}' if experiment_num is not None else ''
        filename = f'trajectory_{result.function_name}_{result.method}{experiment_suffix}.png'

        plt.savefig(os.path.join(save_dir, filename), dpi=300, bbox_inches='tight')
        plt.close()

    @staticmethod
    def plot_convergence(results: Dict[str, OptimizationResult], save_dir: str, function_name: str):
        """Plot convergence with enhanced information"""
        if not results:  # Skip if no results
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Get global minimum if available
        _, f_min = TestFunctions.get_global_minimum(function_name)
        f_min_text = f"(Global min: {f_min})" if f_min is not None else ""

        # Function value convergence
        for method, result in results.items():
            if result and hasattr(result, 'f_path') and result.f_path:  # Check if result and f_path exist
                ax1.semilogy(result.f_path, label=f"{method}")
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Function Value (log scale)')
        ax1.set_title(f'Function Value Convergence {f_min_text}')
        ax1.legend()
        ax1.grid(True)

        # Gradient norm convergence
        for method, result in results.items():
            if result and hasattr(result, 'grad_norm_path') and result.grad_norm_path:  # Check if result and grad_norm_path exist
                ax2.semilogy(result.grad_norm_path, label=f"{method}")
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Gradient Norm (log scale)')
        ax2.set_title('Gradient Norm Convergence')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f'convergence_{function_name}.png'), dpi=300)
        plt.close()

    @staticmethod
    def plot_computational_metrics(results: Dict[str, OptimizationResult], save_dir: str):
        """Plot computational metrics over time"""
        if not results:  # Skip if no results
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Memory usage over time
        for method, result in results.items():
            if result and hasattr(result, 'timestamps') and hasattr(result, 'memory_usage'):
                if result.timestamps and result.memory_usage:  # Check if data exists
                    ax1.plot(result.timestamps, result.memory_usage, label=method)
        ax1.set_xlabel('Time (seconds)')
        ax1.set_ylabel('Memory Usage (MB)')
        ax1.set_title('Memory Usage Over Time')
        ax1.legend()
        ax1.grid(True)

        # FLOPS over time
        for method, result in results.items():
            if result and hasattr(result, 'timestamps') and hasattr(result, 'flops_per_step'):
                if result.timestamps and result.flops_per_step:  # Check if data exists
                    cumulative_flops = np.cumsum(result.flops_per_step)
                    ax2.plot(result.timestamps, cumulative_flops, label=method)
        ax2.set_xlabel('Time (seconds)')
        ax2.set_ylabel('Cumulative FLOPS')
        ax2.set_title('Computational Cost Over Time')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'computational_metrics.png'), dpi=300)
        plt.close()

def get_memory_usage() -> float:
    """Get current memory usage in MB"""
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024

def run_optimization(f: Callable,
                    grad: Callable,
                    hess: Callable,
                    x0: np.ndarray,
                    method: str,
                    function_name: str) -> OptimizationResult:
    """Enhanced optimization runner with detailed metrics"""
    start_time = time.time()
    flop_counter = FLOPCounter()

    # Calculate initial metrics
    f_initial = f(x0)
    grad_initial = grad(x0)

    # Storage for metrics
    path = [x0.copy()]  # Start with initial point
    f_path = [f_initial]
    grad_norm_path = [np.linalg.norm(grad_initial)]
    timestamps = [0.0]
    memory_usage = [get_memory_usage()]
    flops_per_step = [0]

    def callback(xk):
        current_time = time.time() - start_time

        # Calculate metrics
        f_val = f(xk)
        grad_val = grad(xk)
        grad_norm = np.linalg.norm(grad_val)

        # Store metrics
        path.append(xk.copy())
        f_path.append(f_val)
        grad_norm_path.append(grad_norm)
        timestamps.append(current_time)
        memory_usage.append(get_memory_usage())
        flops_per_step.append(flop_counter.flops)

    try:
        # Run optimization with method-specific settings
        if method == 'BFGS':
            result = minimize(f, x0, method=method, jac=grad, callback=callback)
        elif method == 'newton-cg':
            result = minimize(f, x0, method=method, jac=grad, hess=hess, callback=callback)
        elif method in ['trust-exact', 'trust-krylov']:
            result = minimize(f, x0, method=method, jac=grad, hess=hess, callback=callback)
        else:
            raise ValueError(f"Unsupported method: {method}")

        # Calculate final gradient
        grad_final = grad(result.x)

        return OptimizationResult(
            x_final=result.x,
            f_final=result.fun,
            success=result.success,
            iterations=result.nit,
            runtime=time.time() - start_time,
            path=path,
            f_path=f_path,
            grad_norm_path=grad_norm_path,
            timestamps=timestamps,
            memory_usage=memory_usage,
            flops_per_step=flops_per_step,
            method=method,
            dimension=len(x0),
            function_name=function_name,
            x_initial=x0,
            f_initial=f_initial,
            grad_initial=grad_initial,
            grad_final=grad_final
        )

    except Exception as e:
        print(f"Optimization failed: {e}")
        return None

In [23]:
class ExperimentManager:
    """Manages multiple optimization experiments"""
    def __init__(self,
                 test_functions: Dict[str, Tuple[Callable, Callable, Callable]],
                 methods: List[str],
                 dimensions: List[int],
                 n_experiments: int = 50,
                 min_dist: float = 100,
                 max_dist: float = 1000):
        self.test_functions = test_functions
        self.methods = methods
        self.dimensions = dimensions
        self.n_experiments = n_experiments
        self.min_dist = min_dist
        self.max_dist = max_dist

    def generate_starting_points(self, dimension: int, seed: int = None) -> np.ndarray:
        """Generate random starting points with specified distance from origin"""
        if seed is not None:
            np.random.seed(seed)

        starting_points = []

        for _ in range(self.n_experiments):
            # Generate random direction vector
            direction = np.random.randn(dimension)
            direction = direction / np.linalg.norm(direction)

            # Generate random distance within specified range
            distance = np.random.uniform(self.min_dist, self.max_dist)

            # Create starting point
            point = direction * distance
            starting_points.append(point)

        return np.array(starting_points)

    def run_experiments(self, base_dir: str = "optimization_results"):
        """Run all experiments with proper directory structure"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        experiment_dir = os.path.join(base_dir, f"experiment_{timestamp}")

        # Store all results for final analysis
        all_results = []

        for func_name, (f, grad, hess) in self.test_functions.items():
            print(f"\nTesting {func_name} function:")

            for dim in self.dimensions:
                print(f"\nDimension: {dim}")

                # Generate starting points for this dimension
                starting_points = self.generate_starting_points(dim, seed=42)

                # Create directories
                dim_dir = os.path.join(experiment_dir, func_name, f"{dim}D")
                results_dir = os.path.join(dim_dir, "results")
                trajectory_dir = os.path.join(dim_dir, "trajectories")
                os.makedirs(results_dir, exist_ok=True)
                if dim == 2:
                    os.makedirs(trajectory_dir, exist_ok=True)

                # Save starting points
                pd.DataFrame(starting_points).to_csv(
                    os.path.join(dim_dir, "starting_points.csv"),
                    index=False
                )

                # Run optimization for each starting point
                for i, x0 in enumerate(starting_points):
                    print(f"\nExperiment {i+1}/{self.n_experiments}")
                    experiment_results = {}

                    for method in self.methods:
                        print(f"Running {method}...")
                        try:
                            result = run_optimization(f, grad, hess, x0, method, func_name)
                            if result is not None:
                                experiment_results[method] = result

                                # Add to results list
                                all_results.append({
                                    'function': func_name,
                                    'dimension': dim,
                                    'experiment': i,
                                    'method': method,
                                    'start_distance': np.linalg.norm(x0),
                                    'final_value': result.f_final,
                                    'iterations': result.iterations,
                                    'runtime': result.runtime,
                                    'success': result.success,
                                    'distance_to_minimum': result.distance_to_minimum,
                                    'f_error': result.f_error,
                                    'initial_gradient_norm': np.linalg.norm(result.grad_initial),
                                    'final_gradient_norm': np.linalg.norm(result.grad_final)
                                })

                        except Exception as e:
                            print(f"Error with {method}: {e}")
                            continue

                    # Generate plots for this experiment
                    if experiment_results:
                        Visualizer.plot_convergence(experiment_results, results_dir,
                                                 f"{func_name}_exp{i}")
                        Visualizer.plot_computational_metrics(experiment_results, results_dir)
                        Visualizer.plot_optimization_summary(experiment_results, results_dir,
                                                         f"{func_name}_exp{i}")

                        if dim == 2:
                          for method, result in experiment_results.items():
                              if result is not None:
                                  Visualizer.plot_2d_trajectory(f, result, trajectory_dir, i)  # Pass experiment number i

                # Generate statistical plots for this dimension
                results_df = pd.DataFrame([r for r in all_results
                                         if r['function'] == func_name and r['dimension'] == dim])
                self._generate_statistics(results_df, dim_dir)

        # Save complete results
        results_df = pd.DataFrame(all_results)
        results_df.to_csv(os.path.join(experiment_dir, "all_results.csv"), index=False)
        self._generate_summary_statistics(results_df, experiment_dir)

    def _generate_statistics(self, results_df: pd.DataFrame, save_dir: str):
        """Generate statistical visualizations for a specific dimension"""
        # Violin plots
        plt.figure(figsize=(12, 6))
        sns.violinplot(data=results_df, x='method', y='final_value')
        plt.title('Distribution of Final Values')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'final_values_distribution.png'))
        plt.close()

        # Success rates
        plt.figure(figsize=(10, 6))
        success_rates = results_df.groupby('method')['success'].mean()
        success_rates.plot(kind='bar')
        plt.title('Success Rates by Method')
        plt.ylabel('Success Rate')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'success_rates.png'))
        plt.close()

    def _generate_summary_statistics(self, results_df: pd.DataFrame, save_dir: str):
        """Generate overall summary statistics"""
        summary = results_df.groupby(['function', 'dimension', 'method']).agg({
            'final_value': ['mean', 'std', 'min', 'max'],
            'runtime': ['mean', 'std'],
            'iterations': ['mean', 'std'],
            'success': 'mean',
            'distance_to_minimum': ['mean', 'std'],
            'f_error': ['mean', 'std']
        }).reset_index()

        summary.to_csv(os.path.join(save_dir, "experiment_summary.csv"))

In [ ]:
def main():
    test_functions = {
        'ackley': (
            TestFunctions.ackley,
            TestFunctions.ackley_gradient,
            TestFunctions.ackley_hessian
        ),
        'rastrigin': (
            TestFunctions.rastrigin,
            TestFunctions.rastrigin_gradient,
            TestFunctions.rastrigin_hessian
        ),
        'sphere': (
            TestFunctions.sphere,
            TestFunctions.sphere_gradient,
            TestFunctions.sphere_hessian
        ),
        'rosenbrock': (
            TestFunctions.rosenbrock,
            TestFunctions.rosenbrock_gradient,
            TestFunctions.rosenbrock_hessian
        )
    }

    methods = ['BFGS', 'newton-cg', 'trust-exact', 'trust-krylov']
    dimensions = [2, 4, 8, 12]

    # Create experiment manager
    experiment = ExperimentManager(
        test_functions=test_functions,
        methods=methods,
        dimensions=dimensions,
        n_experiments=50,
        min_dist=100,
        max_dist=1000
    )

    # Run experiments
    experiment.run_experiments()

if __name__ == "__main__":
    main()


Testing ackley function:

Dimension: 2

Experiment 1/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 2/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 3/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 4/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 5/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 6/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 7/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 8/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 9/50
Running BFGS...
Running newton-cg...
Running trust-exact...
Running trust-krylov...

Experiment 10/50
Running BFGS...
Running newton-cg